File: cash_flow.ipynb\
Author: alexkobz\
Date: 19-11-2024\
Task: DSRFU-277

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date as d, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = pd.to_datetime(last_day_month)
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF, RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [4]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (101024, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,3.0,3.0,NaN,NaN,NaN,RU0006766433,3.0,3.0,0.0,3.0,3.0,NaN,NaN,"Акции обыкновенные АО ""РУСАЛ Новокузнецк""",None,None,001,NaN,0.0,None,0.0,NaN,0.10,RUB,5.399717e+07,1-01-10239-F,2003-12-08T00:00:00,РО ФКЦБ России в Сибирском федеральном округе,2004-01-15T00:00:00,NaN,2004-01-15T00:00:00,None,None,5.399717e+07,NaN,None,RNOV,NaN,5.399717e+07,"АО ""РУСАЛ Новокузнецк""",None,None,None,CORPREG,NaN,None,NaN,NaN,НД,2011-11-19T00:00:00,5938000504,2011-11-19T00:00:00,NaN,None,None,None,None,None,None,None,None,None,None,None,Бездокументарная именная,uncertificated registered,Акции прочих резидентов (обыкновенные),Размещен,Placed,Объединение выпусков,обыкновенные,ordinary,None,None,None,NaN,NaN,NaN,NaN,2.0,2.0,11.0,1.0,None,None,None,S112,"JSC ""RUSAL Novokuznetsk"" ORD SHS",None,4383,None,None,"""РУСАЛ Новокузнецкий Алюминиевый Завод"", АО, а...",None,RU0006766433,РУСАЛ Новокузнецк-1-ао,NaN,1-01-10239-F,2003-12-08T00:00:00,ФКЦБ,NaN,None,None,ESVXFR,None,Акция,None,None,NaN,0.0,NaN,None,None,NaN,RU,NaN,RUB,0.10,NaN,NaN,NaN,В обращении,5.399717e+06,5.399717e+07,5.399717e+06,5.399717e+07,None,None,None,NaN,None,None,None,РУСАЛ Новокузнецк,05785276,Металлургический,6979.0,4221000535,RU,Кемеровская область,None,NaN,2004-01-15T00:00:00,None,NaN,2004-01-15T00:00:00,None,0.0,None,Обыкн-ая,Осн-ой,None,2024-07-26T12:20:27,NaN,На обслуживании,NaN,None,NaN,Исключен с РТС 07.02.2001 г.,NaN,None,None,Non

In [ ]:
Calendar = RuDataDF("CalendarV2").df
Calendar.head()

In [6]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

AccruedInterestOnDate started
AccruedInterestOnDate finished. AccruedInterestOnDate shape (100842, 14)


,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,6583,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '6583': Нет...,None,None,None
1,6582,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '6582': Нет...,None,None,None
2,6556,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '6556': Нет...,None,None,None
3,6533,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '6533': Нет...,None,None,None
4,6534,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '6534': Нет...,None,None,None


In [7]:
FloaterData = RuDataDF("FloaterData").df
FloaterData.head()

FloaterData started
FloaterData finished. FloaterData shape (1749, 10)


,fintoolId,beg_period,end_period,start_point,model_type,date_pub_n,date_pub_type,date_fix_n,date_fix_type,bases
0,11193,2000-12-01T00:00:00,2050-12-01T00:00:00,E,single,NaN,None,10.0,W,"[{'base_no': 1, 'calc_type': 'FIX', 'base_fint..."
1,30316,2015-03-18T00:00:00,2025-09-18T00:00:00,B,max,NaN,None,2.0,W,"[{'base_no': 4, 'calc_type': None, 'base_finto..."
2,30316,2025-09-18T00:00:00,2025-12-18T00:00:00,B,max,NaN,None,2.0,W,"[{'base_no': 5, 'calc_type': 'FIX', 'base_fint..."
3,36325,2009-08-27T00:00:00,2028-09-15T00:00:00,B,single,NaN,None,1.0,W,"[{'base_no': 1, 'calc_type': 'FIX', 'base_fint..."
4,59702,2013-12-11T00:00:00,2028-11-22T00:00:00,B,single,NaN,None,5.0,C,"[{'base_no': 2, 'calc_type': 'FIX', 'base_fint..."


#### Шаг 2. Предобработка таблиц до экспорта в excel

In [8]:
TransformedFintoolReferenceData = FintoolReferenceData[FintoolReferenceData['fintooltype']=='Облигация']\
.copy()[[
    'isincode', 'fintoolid', 'status', 'nickname', 'issuername', 'issuerinn', 'borrowername', 'borrowerinn',
    'fintooltype', 'facevalue', 'begdistdate', 'faceftname', 'nominaltype', 'coupontype', 'floatratename'
]]\
.drop_duplicates(subset=['isincode'])\
.rename(columns={
    'isincode': "isin",
    'fintoolid': "fintool",
    'status': "status",
    'nickname': "name",
    'issuername': "issuer",
    'issuerinn': "issuer_INN",
    'borrowername': "borrower",
    'borrowerinn': "borrower_INN",
    'fintooltype': "type",
    'facevalue': "face_value",
    'begdistdate': "issue_date",
    'faceftname': "currency",
    'nominaltype': "principal_type",
    'coupontype': "coupon_type",
    'floatratename': "rate"
})
TransformedFintoolReferenceData.set_index('isin', drop=True, inplace=True)
TransformedFintoolReferenceData['issue_date'] = pd.to_datetime(TransformedFintoolReferenceData['issue_date'])

In [9]:
TransformedAccruedInterestOnDate = AccruedInterestOnDate.copy().dropna(subset=["currentFaceValue"])
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate\
.merge(FintoolReferenceData[['fintoolid', "isincode"]], how='left', left_on='fintoolId', right_on='fintoolid')\
.drop(columns=['fintoolid'])\
.rename(columns={
    "isincode": "isin", 
    'currentFaceValue': 'face_value', 
    'faceValueCurrency': 'currency'
})
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate[['isin', 'face_value', 'currency']].set_index('isin')

#Добавление загруженных данных в результирующий датафрейм
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate[~TransformedAccruedInterestOnDate.index.duplicated()]
TransformedFintoolReferenceData.update(TransformedAccruedInterestOnDate)

In [10]:
TransformedFloaterData = FloaterData.copy()
TransformedFloaterData = pd\
.concat([TransformedFloaterData, pd.DataFrame([base[0] for base in FloaterData.bases])], axis=1)\
.drop(columns=['bases'])
TransformedFloaterData = TransformedFloaterData\
.merge(FintoolReferenceData[['fintoolid', "isincode"]], how='left', left_on='fintoolId', right_on='fintoolid')\
.drop(columns=['fintoolid'])\
.reset_index(drop=True)
TransformedFloaterData['beg_period'] = pd.to_datetime(TransformedFloaterData['beg_period'])
TransformedFloaterData['end_period'] = pd.to_datetime(TransformedFloaterData['end_period'])
TransformedFloaterData['premium_to_base'] /= 10000

In [201]:
TransformedCalendar = Calendar.copy()
TransformedCalendar.loc[TransformedCalendar['typeOperation'].isin(['O', 'N', 'P']), 'eventType'] = 'PUT'
TransformedCalendar['eventDate'] = pd.to_datetime(TransformedCalendar['eventDate'])
TransformedCalendar = TransformedCalendar[~TransformedCalendar['eventDate'].isna()]
TransformedCalendar.loc[~TransformedCalendar['revisedPay1Bond'].isna(), 'pay1Bond'] = \
TransformedCalendar.loc[~TransformedCalendar['revisedPay1Bond'].isna(), 'revisedPay1Bond']
TransformedCalendar['Currency_temp'] = TransformedCalendar['faceFTName']
TransformedCalendar.loc[~TransformedCalendar['currencyFTName'].isna(), 'Currency_temp'] = \
TransformedCalendar.loc[~TransformedCalendar['currencyFTName'].isna(), 'currencyFTName']
TransformedCalendar.loc[(~TransformedCalendar['paymentCurrencyFTName'].isna())&(~TransformedCalendar['revisedPay1Bond'].isna()), 'Currency_temp'] = \
TransformedCalendar.loc[(~TransformedCalendar['paymentCurrencyFTName'].isna())&(~TransformedCalendar['revisedPay1Bond'].isna()), 'paymentCurrencyFTName']
TransformedCalendar = TransformedCalendar[TransformedCalendar['eventType'].isin(['CPN', 'MTY', 'PUT'])] # если нужны оферты колл, нужно аккуратно подойти к выбору typeOperation
TransformedCalendar['value'] /= 100

In [12]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep

key = 'CurrencyRate'
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
currencies_rudata = []
curr = TransformedCalendar[['Currency_temp', 'faceFTName']].drop_duplicates()
curr = curr[curr['Currency_temp'] != curr['faceFTName']].dropna().reset_index(drop=True)
for _, row in curr.iterrows():
    ans = requests.post(url, json={'from': row['Currency_temp'], 'to': row['faceFTName'], 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
    result: dict = ans.json()
    result["from"] = row['Currency_temp']
    result["to"] = row['faceFTName']
    currencies_rudata.append(result)
    sleep(1)
CurrencyRate = pd.DataFrame(currencies_rudata)
CurrencyRate.rename(columns={"rate": "FX"}, inplace=True)
CurrencyRate.head()

CPU times: user 321 ms, sys: 28.2 ms, total: 350 ms
Wall time: 8.93 s


,FX,endOfDayRate,error,from,to
0,0.010304,0.010307,None,RUB,USD
1,0.009504,0.009477,None,RUB,EUR
2,1.299600,1.299600,None,GBP,USD
3,97.053000,97.022600,None,USD,RUB
4,0.007928,0.007931,None,RUB,GBP


In [202]:
TransformedCalendar = TransformedCalendar.merge(CurrencyRate[["FX", "from", "to"]], how='left', left_on=['Currency_temp', 'faceFTName'], right_on=['from', 'to'])
TransformedCalendar['FX'] = TransformedCalendar['FX'].fillna(1)
TransformedCalendar['CF'] = TransformedCalendar['pay1Bond'] * TransformedCalendar['FX']
currencies = pd.read_csv("./data/Input/currencies.csv", header=0, encoding='cp1251', sep=';').set_index("Currency")["Num"]
TransformedCalendar['Currency'] = TransformedCalendar['faceFTName'].map(currencies).fillna(0).astype(np.int32)

In [203]:
TransformedCalendar = TransformedCalendar[[
    'isiNcode', 'finToolID', 'nickname', 'eventDate', 'eventType', 
    'typeOperation', 'value', 'CF', 'faceFTName', 'note', 'eventTotalVol'
]].rename(columns={
    'isiNcode': "isin",
    'finToolID': "fintool",
    'nickname': "name",
    'eventDate': "payment_date", 
    'eventType': "payment_type", 
    'typeOperation': 'operation_type',
    'value': "rate", 
    'CF': "payment_sum", 
    'faceFTName': "currency", 
    'note': "note",
    'eventTotalVol': 'event_total_vol'
})
TransformedCalendar.dropna(subset="isin", inplace=True)

#### Шаг 3. Корректировка потока

In [511]:
bonds = TransformedCalendar\
.merge(
    TransformedFintoolReferenceData[["face_value", "principal_type", "coupon_type", "issue_date"]], 
    how='left', 
    left_on="isin", 
    right_index=True)\
.rename(columns={
    "face_value": "nominal"
})
bonds["nominal"] = bonds["nominal"].fillna(0)
bonds["rate"] = bonds["rate"].fillna(0)
bonds["payment_sum"] = bonds["payment_sum"].fillna(0)
bonds['event_total_vol'] = bonds['event_total_vol'].fillna(0)
bonds["name"] = bonds["name"].str.replace('[\n\t\r;,]', '.', regex=True)
bonds["note"] = bonds["note"].str.replace('[\n\t\r;,]', '.', regex=True)
bonds["payment_type_NUM"] = bonds["payment_type"].map({"CPN": 1, "MTY": 2, "PUT": 3})
bonds.sort_values(['isin', 'payment_date', 'payment_type_NUM'], inplace=True)
bonds["MTY"] = np.where(bonds.payment_type=="MTY", bonds['rate'], 0)
bonds["paid_nominal"] = bonds.groupby(["isin"])["MTY"].cumsum()
bonds["remain_nominal"] = bonds["nominal"]*(1 - bonds["paid_nominal"])
bonds["prev_cpn"] = bonds["isin"].map(
    bonds[
        (bonds['payment_date']<=REPORT_DATE)&
        (bonds['payment_type']=='CPN')
    ].groupby('isin')\
    .last()["rate"]\
    .to_dict()
)
bonds['prev_date'] = bonds.groupby('isin')['payment_date'].shift(1)
bonds['days_diff'] = (bonds['payment_date'] - bonds['prev_date']).dt.days

если у нас perpetual, то есть нет даты погашения, добавляем в конец прогнозных потоков погашение

In [493]:
last_noMTY = bonds[
    ~bonds['isin'].isin(bonds[(bonds["payment_type"].isin(["MTY"]))&(~bonds['payment_date'].isna())]['isin'])]\
.groupby('isin')\
.last()\
.reset_index()
last_noMTY['payment_type'] = 'MTY'
last_noMTY['payment_type_NUM'] = 2
last_noMTY['payment_sum'] = last_noMTY["isin"]\
.map(
    bonds\
    .groupby('isin')\
    .last()
    ["remain_nominal"]\
    .to_dict()
)
bonds = pd.concat(
    [bonds, last_noMTY], axis=0)\
.sort_values(['isin', 'payment_date', 'payment_type_NUM'])\
.reset_index(drop=True)

в базе по досрочно погашенным выпускам может стоять и оферта, и погашение двумя строчками. Оставляем только одну с купоном.

In [495]:
bonds['put'] = bonds['payment_type']\
.eq('PUT')\
.groupby(bonds['isin'])\
.cumsum()\
.shift(1)\
.fillna(0)
bonds = bonds[bonds['put']==0]
bonds['prev_isin'] = bonds['isin'].shift(1)
bonds['prev_date'] = bonds['payment_date'].shift(1)
bonds['prev_type'] = bonds['payment_type'].shift(1)
bonds = bonds[
    ~(
        (bonds['isin'] == bonds['prev_isin']) & 
        (bonds['payment_date'] == bonds['prev_date']) & 
        (bonds['payment_type'] == 'PUT') & 
        (bonds['prev_type'] == 'MTY')
    )
]

Корректируем поток для облигаций с индексируемым номиналом

In [419]:
indexed_bonds_isins = bonds[bonds['principal_type']=='Инфляционно-Индексируемый']['isin']
indexed_bonds = bonds[bonds['isin'].isin(indexed_bonds_isins)]
not_indexed_bonds = bonds[~bonds['isin'].isin(indexed_bonds_isins)]
cpn_pay_mask = indexed_bonds['payment_type']=='CPN'
indexed_bonds.loc[cpn_pay_mask,'payment_sum'] = indexed_bonds.loc[cpn_pay_mask,'rate']*indexed_bonds.loc[cpn_pay_mask,'remain_nominal']*indexed_bonds.loc[cpn_pay_mask,'days_diff']/365
bonds = pd.concat([indexed_bonds, not_indexed_bonds], axis=0).sort_values(['isin', 'payment_date', 'payment_type_NUM']).reset_index(drop=True)

оставляем только будущие потоки

In [498]:
bonds = bonds[bonds['payment_date']>=REPORT_DATE]

Деление на флоатеры и обычные облигации

In [422]:
Bonds = bonds[~bonds['fintool'].isin(FloaterData["fintoolId"])]
floaters = bonds[bonds['fintool'].isin(FloaterData["fintoolId"])]

Флоатеры

In [2]:
import sqlite3

conn = sqlite3.connect(":memory:") 

In [3]:
TransformedFloaterData.to_sql("TransformedFloaterData", conn, index=False)
floaters.to_sql("floaters", conn, index=False)

NameError: name 'TransformedFloaterData' is not defined

In [445]:
Floaters = pd.read_sql_query("""
    SELECT 
        b.*
        , fl.beg_period
        , fl.end_period
        , fl.premium_to_base
        , fl.start_point
    FROM floaters b
    LEFT OUTER JOIN TransformedFloaterData fl ON b.fintool = fl.fintoolId AND b.payment_date BETWEEN fl.beg_period AND fl.end_period
    """
    , conn
)
floaters_fixed_dict = Floaters.groupby('isin').first().to_dict()
fix_cpn_mask = (Floaters['payment_type']=='CPN') & (Floaters['operation_type']=='Фиксированный') 
Floaters['fix_rate'] = Floaters['isin'].map(floaters_fixed_dict['rate'])
Floaters['fix_cf'] = Floaters['isin'].map(floaters_fixed_dict['payment_sum'])
Floaters.loc[fix_cpn_mask, 'rate'] = Floaters.loc[Floaters['operation_type']=='Фиксированный', 'fix_rate']
Floaters.loc[fix_cpn_mask, 'payment_sum'] = Floaters.loc[Floaters['operation_type']=='Фиксированный', 'fix_cf']
base_mask = (Floaters['payment_type']=='CPN') & (Floaters['payment_date']>Floaters['beg_period']) & (Floaters['payment_date']<=Floaters['end_period'])
Floaters.loc[base_mask, 'base_period'] = Floaters.loc[base_mask, 'premium_to_base']
Floaters['base'] = Floaters[['premium_to_base', 'fix_rate']].max(axis=1)
cpn_pay_nan_mask = (Floaters['payment_type']=='CPN') & (Floaters['payment_sum']==0) & (~Floaters['base'].isna())
Floaters.loc[base_mask, 'cf'] = Floaters.loc[cpn_pay_nan_mask, 'base']*Floaters.loc[cpn_pay_nan_mask,'remain_nominal']*Floaters.loc[cpn_pay_nan_mask,'days_diff']/365
Floaters.loc[base_mask, 'payment_sum'] = Floaters.loc[base_mask, 'cf'].combine_first(Floaters.loc[base_mask, 'payment_sum'])
Floaters.loc[base_mask, 'rate'] = Floaters.loc[base_mask, 'base'].combine_first(Floaters.loc[base_mask, 'rate'])

#### Шаг 4. Объединение облигаций с постоянным и плавающим купоном в один датафрейм

In [394]:
Bonds["is_floater"] = False
Floaters["is_floater"] = True

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_1181/4080972275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bonds["is_floater"] = False


In [524]:
bonds = pd.concat([Bonds, Floaters], axis=0).sort_values(['isin', 'payment_date', 'payment_type_NUM']).reset_index(drop=True)
bonds.loc[bonds['payment_type']=='MTY', 'rate'] = bonds.loc[bonds['payment_type']=='MTY', 'nominal'] / bonds.loc[bonds['payment_type']=='MTY', 'payment_sum']

In [525]:
bonds['id_payment'] = bonds.groupby(['isin']).cumcount() + 1
bonds["report_date"] = pd.to_datetime(REPORT_DATE)
bonds["payment_date"] = pd.to_datetime(bonds["payment_date"])

Мэппинг имен и кодов валют

In [526]:
currencies = pd.read_csv("./data/Input/currencies.csv", header=0, encoding='cp1251', sep=';').set_index("Currency")["Num"]
bonds["currency"] = bonds["currency"].map(currencies).fillna(0).astype(np.int32)

Добавление времени выгрузки и выбор колонок

In [527]:
bonds["export_timestamp"] = dt.now().strftime("%Y%m%d%H%M%S")

In [528]:
bonds = bonds[
    ["isin", "fintool", "name", "payment_date", "payment_type", "rate", "payment_sum", "currency", "report_date", "export_timestamp"]
]

In [529]:
bonds.to_parquet(f'./data/Output/cash_flows.parquet', index=False)

##### Шаг 4.2. Write excel

In [1]:
from functions.send_email import main
main("cash_flows.parquet")